# Unnest the alert data file and merge it with the notification file
In this notebook we will parse the alert file which is a json and merge this file with the notifications to have one new csv.

In [1]:
# Import general packages
import pandas as pd
import numpy as np
import statistics
pd.set_option('max_columns', None) #Shows all columns always! No ... inbetween columns.

# Processing json file
import json

# Switch off warnings
import warnings 
warnings.filterwarnings('ignore')

## Parsing the json file to a pandas dataframe

In [2]:
# Load json file
with open('../data/alerts_20200615-20201119.json') as data_file:
    data = json.load(data_file)

In [3]:
# Create dataframe and extract first level of json file
alerts = pd.json_normalize(data['alerts'], max_level=0)

In [4]:
# Glance into the dataframe 
alerts.head(1)

,dateCreated,datePublished,dateClosed,affectedTransport,locationSelection,reportId,feedId,cause,effect,description
0,2020-06-15T18:15:58.500Z,2020-06-15T18:26:44.598Z,2020-06-15T19:10:00.776Z,{'IFCZKKX7-k6Z96u1AIhqQg': {'6bE4GjRfgUanCKu1A...,"{""type"":""FeatureCollection"",""features"":[{""type...",501468,TDxLYyiCXYN0YxYBDYjj,March,CirculationShutdown,Cierre vial en José María Izazaga de José Marí...


In [5]:
print(alerts.shape)
print(alerts.columns)

(4495, 10)
Index(['dateCreated', 'datePublished', 'dateClosed', 'affectedTransport',
       'locationSelection', 'reportId', 'feedId', 'cause', 'effect',
       'description'],
      dtype='object')


In [6]:
# The column 'affectedTransport' contains several dictionaries
agencies = alerts["affectedTransport"]

In [7]:
# Get the transport agencies along keys from the transport dictionary
def get_agencies(transport_dict):
    return list(sorted(transport_dict.keys()))

In [8]:
# Get the nested lines along the keys
def get_lines_nested(transport_dict):
    lines = []
    for k, v in sorted(transport_dict.items()): 
        # now our v is another .. dict with line name and items .. but as there are more lines than agiencies.. this will need to be nested list
        lines.append(sorted(v.keys()))
    return lines

In [9]:
# Get the metadata nested in the lines along the keys
label = "closedState" # publishedId, publishedState, stopId
def get_metadata_nested(label, transport_dict):
    metadata = []
    for k, v in sorted(transport_dict.items()): 
    # k is agency, v is line and we have more lines per agency
        metametadata = []
        for kk, vv in sorted(v.items()):
            # these are the values we want
            # value = d.get(key, "empty")
            metametadata.append(vv.get(label, None))
        metadata.append(metametadata)
    return metadata

In [10]:
# Extract agencies from 'affectedTransport' into Pandas dataframe alerts
alerts['agencies'] = alerts['affectedTransport'].apply(lambda x: get_agencies(x))

# Creation of nested variables into the dataframe Pandas dataframe alerts
alerts['lines_nested'] = alerts['affectedTransport'].apply(lambda x: get_lines_nested(x))
alerts['publishedIds_nested'] = alerts['affectedTransport'].apply(lambda x: get_metadata_nested('publishedId',x))

# Print columns
alerts.columns

Index(['dateCreated', 'datePublished', 'dateClosed', 'affectedTransport',
       'locationSelection', 'reportId', 'feedId', 'cause', 'effect',
       'description', 'agencies', 'lines_nested', 'publishedIds_nested'],
      dtype='object')

In [11]:
# The dataframe's dictionaries contain several nested lists, e.g. see alert No. 3
print('*Agencies: ', alerts["agencies"][3])
print('*Nested lines: ', alerts["lines_nested"][3]) #lines nested in agencies
print('*publishedIds_nested: ', alerts["publishedIds_nested"][3]) #publisedIDs nested in lines

*Agencies:  ['GtvOEQAFZ0GtU6u4AXwvPg', 'JUR9bFXmVkWDHqu4AXaY0g', 'JfA8Bw8Zp024Kqu4AXiSpQ']
*Nested lines:  [['7Kq2iXAEkUuXWau4AXww3Q', 'I9INYshyhEinHqu4AXwwzw', 'iMzaKgmc8EiJ_6u4AXwwoQ', 'jT3h9UcgUkeURKu4AXwv3Q', 'vBhN2IRVp0azE6u4AXwwWg'], ['_a2kY7D-6EqBHKu4AXaZiw'], ['2M6bixGmNUa206u4AXiSuA']]
*publishedIds_nested:  [['VIM11MpZ3EmIgJgPR5ujcA', 'zrOeGJz5m0qy4yt66QvIVA', 'BZHFQ9rZPU6Vy7822TT2WQ', 'LYQyD5ROV0SAHBQMA5ehYQ', 'g_lc2vUk-EynCFEY8qqSNw'], ['dCHaFlmaUEaPtB626diqRQ'], ['jWddj92aBUSohqCTLQCMZg']]


In [12]:
# Assign a unique index ID to each row of the dataset
alerts['alert_index'] = np.arange(alerts.shape[0])
print(alerts.columns)
alerts.head(2)

Index(['dateCreated', 'datePublished', 'dateClosed', 'affectedTransport',
       'locationSelection', 'reportId', 'feedId', 'cause', 'effect',
       'description', 'agencies', 'lines_nested', 'publishedIds_nested',
       'alert_index'],
      dtype='object')


,dateCreated,datePublished,dateClosed,affectedTransport,locationSelection,reportId,feedId,cause,effect,description,agencies,lines_nested,publishedIds_nested,alert_index
0,2020-06-15T18:15:58.500Z,2020-06-15T18:26:44.598Z,2020-06-15T19:10:00.776Z,{'IFCZKKX7-k6Z96u1AIhqQg': {'6bE4GjRfgUanCKu1A...,"{""type"":""FeatureCollection"",""features"":[{""type...",501468,TDxLYyiCXYN0YxYBDYjj,March,CirculationShutdown,Cierre vial en José María Izazaga de José Marí...,"[GtvOEQAFZ0GtU6u4AXwvPg, IFCZKKX7-k6Z96u1AIhqQ...","[[m1ukaXkjLkaCiau4AXwvzg], [6bE4GjRfgUanCKu1AI...","[[O26r3Zng20KQMs4b5uOoig], [YtC9yZBWsEW0H2JTWy...",0
1,2020-06-15T18:26:38.579Z,2020-06-15T18:26:47.264Z,2020-06-15T18:35:01.496Z,{'JUR9bFXmVkWDHqu4AXaY0g': {'Ugvp4KWhfE-5eKu4A...,"{""type"":""FeatureCollection"",""features"":[{""type...",1272592149247188993,9wuROlKTul0dN6uiZUae,March,Delays,Manifestantes provenientes de José María Pino ...,[JUR9bFXmVkWDHqu4AXaY0g],[[Ugvp4KWhfE-5eKu4AXaZLg]],[[1MiSoiLFUUaL6PUDwMEUkA]],1


In [13]:
# Drop no longer needed columns in alerts dataframe and create a new dataframe alerts_trimmed
alerts_trimmed = alerts.drop(['agencies', 'lines_nested', 'affectedTransport', 'publishedIds_nested'], axis=1)
print(alerts_trimmed.columns)

Index(['dateCreated', 'datePublished', 'dateClosed', 'locationSelection',
       'reportId', 'feedId', 'cause', 'effect', 'description', 'alert_index'],
      dtype='object')


In [14]:
# Detangle the nested lists and create a new dataframe df_detangled
rows_list1 = []

for row_index, row in alerts.iterrows():
    alert_index = row['alert_index']
    agency_count = len(row['agencies'])
    for i in range(agency_count):    
        agency = row['agencies'][i]
        lines = row['lines_nested'][i]
        publishedIds = row['publishedIds_nested'][i]
        lines_count = len(lines)
        for i2 in range(lines_count):
            line = lines[i2]
            publishedId = publishedIds[i2]

            dict1 = {}
            dict1['alert_index'] = alert_index
            dict1['agency'] =  agency
            dict1['line'] = line
            dict1['publishedId'] = publishedId

            rows_list1.append(dict1)

df_detangled= pd.DataFrame(rows_list1)

In [15]:
# Merge the df_detangled and the alerts_trimmed dataframes based on 'alert_index'
alerts_exploded = pd.merge(alerts_trimmed, df_detangled, on = 'alert_index')
print(alerts_exploded.columns)
print(alerts_exploded.shape)

Index(['dateCreated', 'datePublished', 'dateClosed', 'locationSelection',
       'reportId', 'feedId', 'cause', 'effect', 'description', 'alert_index',
       'agency', 'line', 'publishedId'],
      dtype='object')
(33087, 13)


## Merge the original json- and the csv-file

In [16]:
# Merge the two csv files
# Rename column
alerts_exploded.rename(columns={'publishedId': 'AlertID'}, inplace=True)

# Export dataframe as csv file
alerts_exploded.to_csv(r'../data/alerts_exploded.csv', index = False)

# Merge the two csv files based on column 'AlertID'
df_notifications = pd.read_csv("../data/notifications_202009-202011.csv")
df_alerts_exploded = pd.read_csv("../data/alerts_exploded.csv")
df = df_notifications.merge(df_alerts_exploded, on="AlertID").fillna("")
df.to_csv(r'../data/df_merged.csv', index=False)

In [17]:
# Check shape of dataframes
print(df_notifications.shape)
print(df_alerts_exploded.shape)
print(df.shape)

(30012, 6)
(33087, 13)
(25508, 18)


In total we should have 30012 rows of data, but when merging we only have 25508 rows. In total 7579 AlertIDs/PublishedIDs of the two dataframes do not match. 

In [18]:
df.columns

Index(['device', 'AlertID', 'event_date', 'event_timestamp', 'event_name',
       'version', 'dateCreated', 'datePublished', 'dateClosed',
       'locationSelection', 'reportId', 'feedId', 'cause', 'effect',
       'description', 'alert_index', 'agency', 'line'],
      dtype='object')